# In The Beginning, There Were Relational Databases...
## (Actually in the beginning, there were graph databases.)
![Neo4J](https://d1q6f0aelx0por.cloudfront.net/product-logos/2cd4f81a-e285-4ede-9d9f-99c2349bdd4b-neo4j.png)
# Relational Databases date back to [Edgar F. Codd](https://en.wikipedia.org/wiki/Edgar_F._Codd) in the [1970s](https://pdfs.semanticscholar.org/7dc5/b9737e3f6b69dc896e79e28e20f8537745ba.pdf).

## Relational databases are made of tables.
## The set of table columns is called a schema.
![tables](https://docs.oracle.com/cd/E13167_01/aldsp/docs25/xquery/wwimages/rdbms_tables_relational_wrapper_01.gif)

## The standard for relational databases is Structured Query Language (SQL)

## Free and Open-Source SQL Implementations:
![SQL](http://www.mimastech.com/wp-content/uploads/2016/09/postgresql-mysql-mariadb.png)

# [SQLite](https://www.sqlite.org/) is a library written in C, not a database system.
![SQLite](https://upload.wikimedia.org/wikipedia/commons/3/38/SQLite370.svg)
# It's included in:
# - Most web browsers
# - Most mobile phones
# - Most languages, including Python

## SQL isn't the only game in town...
![Seven Databases in Seven Weeks](http://greyna.eu/portfolio/img/seven-databases-in-seven-weeks-cover.jpg)

## [MongoDB is Web Scale! https://youtu.be/HdnDXsqiPYo](https://youtu.be/HdnDXsqiPYo)
![MongoDB is Web Scale!](https://i.ytimg.com/vi/HdnDXsqiPYo/hqdefault.jpg)

# Objectives:
## - Create databases and  tables
## - Push data into the database
## - Query the data
## - Filter the data

In [1]:
import os
import re
import sqlite3

from bs4 import BeautifulSoup
import requests

## Scraping election data from [Wikipedia](https://en.wikipedia.org/wiki/List_of_MPs_elected_in_the_United_Kingdom_general_election,_2017):

In [2]:
def getWikiMPs(year):
    url = ''.join(['https://en.wikipedia.org/wiki/List_of_MPs_elected_in_the_United_Kingdom_general_election,_',
        str(year)])
    req = requests.get(url)
    assert req.status_code == 200
    table = BeautifulSoup(req.content, "html5lib").find_all("table", class_="wikitable sortable")[0]
    for row in table.find_all("tr")[1:]:
        cells = list(row.find_all('td'))
        constituency = cells[0].text
        incumbent_party = cells[2].text
        member_returned = cells[4].find('a').text
        member_chunks = cells[4].text.split('\xa0')
        try:
            member_party = member_chunks[1][1:-1]
        except:
            member_party = ''
        result_chunks = re.split('[.;,]',cells[5].text)
        outcome = result_chunks[0]
        try:
            notes = result_chunks[1]
        except:
            notes = ''
        yield {'constituency':constituency, 'incumbent_party':incumbent_party, 'member_returned':member_returned,
            'member_party':member_party, 'outcome':outcome, 'notes':notes}

# We get the data as a list of dicts:

In [3]:
ge_2017 = list(getWikiMPs(2017))
ge_2017[0:3]

[{'constituency': 'Aberavon',
  'incumbent_party': 'Labour',
  'member_party': 'L',
  'member_returned': 'The Hon. Stephen Kinnock',
  'notes': '',
  'outcome': 'Seat held'},
 {'constituency': 'Aberconwy',
  'incumbent_party': 'Conservative',
  'member_party': 'C',
  'member_returned': 'Guto Bebb',
  'notes': '',
  'outcome': 'Seat held'},
 {'constituency': 'Aberdeen North',
  'incumbent_party': 'SNP',
  'member_party': 'SNP',
  'member_returned': 'Kirsty Blackman',
  'notes': '',
  'outcome': 'Seat held'}]

## Columns for General Election tables:

In [4]:
election_columns = sorted(list(ge_2017[0].keys())) # The order of dict keys can be tricky.
election_columns

['constituency',
 'incumbent_party',
 'member_party',
 'member_returned',
 'notes',
 'outcome']

## To get a SQLite database, we point "sqlite3.connect" at a file.
## All the operations on our database are done via a "cursor".
## Read the [standard library documentation](https://docs.python.org/3/library/sqlite3.html).

In [5]:
os.remove('elections.sqlite')
election_db = sqlite3.connect('elections.sqlite')
election_cursor = election_db.cursor()

# All database operations are done by passing SQL code to "cursor.execute".
## If we modify the database, we need to call "connection.commit()"
## We mostly get data out with "cursor.fetchall()".

In [6]:
create_statement = '''CREATE TABLE ge_2017(id INTEGER PRIMARY KEY, constituency TEXT,
    incumbent_party TEXT, member_party TEXT, member_returned TEXT, notes TEXT, 
    outcome TEXT)'''
election_cursor.execute(create_statement)
election_db.commit()

# Sometimes we write code that generates SQL for us:

In [7]:
def textTableCreator(name, columns=election_columns):
    cols =  ', '.join(['id INTEGER PRIMARY KEY'] + [' '.join([col,'TEXT']) for col in columns])
    return 'CREATE TABLE '+name+'('+cols+')'
    
textTableCreator('ge_2015')

'CREATE TABLE ge_2015(id INTEGER PRIMARY KEY, constituency TEXT, incumbent_party TEXT, member_party TEXT, member_returned TEXT, notes TEXT, outcome TEXT)'

## To *properly* inject values into SQL, use ?s as place-holders, and pass in a tuple.

In [8]:
def pushWikiMPs(db, cursor, table, data):
    rows = [(i,) + tuple([row[col] for col in election_columns]) for i,row in enumerate(data)]
    cursor.executemany('INSERT INTO '+table+' VALUES (?,?,?,?,?,?,?)', rows)
    db.commit()

In [9]:
pushWikiMPs(election_db, election_cursor, 'ge_2017', ge_2017)

## Using "SELECT" to get all the rows from a table:

In [10]:
election_cursor.execute('SELECT * FROM ge_2017')
election_cursor.fetchall()[0:3]

[(0, 'Aberavon', 'Labour', 'L', 'The Hon. Stephen Kinnock', '', 'Seat held'),
 (1, 'Aberconwy', 'Conservative', 'C', 'Guto Bebb', '', 'Seat held'),
 (2, 'Aberdeen North', 'SNP', 'SNP', 'Kirsty Blackman', '', 'Seat held')]

## Returning a single column:

In [11]:
election_cursor.execute('SELECT incumbent_party FROM ge_2017')
election_cursor.fetchall()[0:8]

[('Labour',),
 ('Conservative',),
 ('SNP',),
 ('SNP',),
 ('SNP',),
 ('Conservative',),
 ('Conservative',),
 ('Conservative',)]

## Using DISTINCT to get the name of each party:

In [12]:
election_cursor.execute('SELECT DISTINCT incumbent_party FROM ge_2017')
election_cursor.fetchall()

[('Labour',),
 ('Conservative',),
 ('SNP',),
 ('Plaid Cymru',),
 ('Labour Co-operative',),
 ('DUP',),
 ('SDLP',),
 ('Sinn Féin',),
 ('Green Party',),
 ('The Speaker',),
 ('Liberal Democrats',),
 ('Independent[a]',),
 ('UUP',),
 ('Independent',)]

## Abbreviations of parties in alphabetical order:

In [13]:
election_cursor.execute('SELECT DISTINCT member_party FROM ge_2017 ORDER BY member_party')
election_cursor.fetchall()

[('C',),
 ('DUP',),
 ('Green',),
 ('Ind',),
 ('L',),
 ('L Co-op',),
 ('LD',),
 ('PC',),
 ('SF',),
 ('SNP',),
 ('Speaker',)]

## Using a "WHERE" clause to get Liberal Democrat seats:

In [14]:
election_cursor.execute('''SELECT constituency, member_returned FROM ge_2017 
    WHERE member_party="LD"''')
election_cursor.fetchall()

[('Bath', 'Wera Hobhouse'),
 ('Caithness, Sutherland and Easter Ross', 'Jamie Stone'),
 ('Carshalton and Wallington', 'Tom Brake'),
 ('East Dunbartonshire', 'Jo Swinson'),
 ('Eastbourne', 'Stephen Lloyd'),
 ('Edinburgh West', 'Christine Jardine'),
 ('Kingston and Surbiton', 'Sir Ed Davey'),
 ('North Norfolk', 'Norman Lamb'),
 ('Orkney and Shetland', 'Alistair Carmichael'),
 ('Oxford West and Abingdon', 'Layla Moran'),
 ('Twickenham', 'Sir Vince Cable'),
 ('Westmorland and Lonsdale', 'Tim Farron')]

## Seats gained BY the Liberal Democrats:

In [15]:
election_cursor.execute('''SELECT constituency, member_returned FROM ge_2017
    WHERE member_party="LD" AND outcome="Seat gain"''')
election_cursor.fetchall()

[('Bath', 'Wera Hobhouse'),
 ('Caithness, Sutherland and Easter Ross', 'Jamie Stone'),
 ('East Dunbartonshire', 'Jo Swinson'),
 ('Eastbourne', 'Stephen Lloyd'),
 ('Edinburgh West', 'Christine Jardine'),
 ('Kingston and Surbiton', 'Sir Ed Davey'),
 ('Oxford West and Abingdon', 'Layla Moran'),
 ('Twickenham', 'Sir Vince Cable')]

## Seats gained FROM the Liberal Democrats:

In [16]:
election_cursor.execute('''SELECT constituency, member_returned, member_party FROM ge_2017
    WHERE incumbent_party="Liberal Democrats" AND outcome="Seat gain"''')
election_cursor.fetchall()

[('Ceredigion', 'Ben Lake', 'PC'),
 ('Leeds North West', 'Alex Sobel', 'L Co-op'),
 ('Richmond Park', 'Zac Goldsmith', 'C'),
 ('Sheffield Hallam', "Jared O'Mara", 'L'),
 ('Southport', 'Damien Moore', 'C')]

## Seats gained from the Conservatives, in party order:

In [17]:
def seatsGainedFrom(cursor,party):
    cursor.execute('SELECT constituency, member_returned, member_party FROM ge_2017 '
        +'WHERE incumbent_party=? AND outcome="Seat gain" ORDER BY member_party', (party,))
    return cursor.fetchall()
seatsGainedFrom(election_cursor,'Conservative')[0:10]

[('Battersea', 'Marsha de Cordova', 'L'),
 ('Bedford', 'Mohammad Yasin', 'L'),
 ('Bristol North West', 'Darren Jones', 'L'),
 ('Bury North', 'James Frith', 'L'),
 ('Canterbury', 'Rosie Duffield', 'L'),
 ('Cardiff North', 'Anna McMorrin', 'L'),
 ('Colne Valley', 'Thelma Walker', 'L'),
 ('Crewe and Nantwich', 'Laura Smith', 'L'),
 ('Croydon Central', 'Sarah Jones', 'L'),
 ('Derby North', 'Chris Williamson', 'L')]

## Using "COUNT" to return how many seats were lost by the Conservatives:

In [18]:
election_cursor.execute('''SELECT COUNT(*) FROM ge_2017
    WHERE incumbent_party="Conservative" AND outcome="Seat gain" ORDER BY member_party''')
election_cursor.fetchall()

[(33,)]

## Using "GROUP BY" to return how many seats each party gained from the Conservatives:

In [19]:
election_cursor.execute('''SELECT member_party, COUNT(*) FROM ge_2017
    WHERE incumbent_party="Conservative" AND outcome="Seat gain" GROUP BY member_party''')
election_cursor.fetchall()

[('L', 25), ('L Co-op', 3), ('LD', 5)]

## Using "GROUP BY" to see who won the election:

In [20]:
election_cursor.execute('''SELECT member_party, COUNT(*) AS seats FROM ge_2017
    GROUP BY member_party
    ORDER BY seats DESC''')
result_2017 = election_cursor.fetchall()
result_2017

[('C', 317),
 ('L', 226),
 ('L Co-op', 36),
 ('SNP', 35),
 ('LD', 12),
 ('DUP', 10),
 ('SF', 7),
 ('PC', 4),
 ('Green', 1),
 ('Ind', 1),
 ('Speaker', 1)]

## (Arguably, the answer is the DUP...)

In [21]:
print('Conservatives: ', result_2017[0][1], "Everyone Else: ", sum([r[1] for r in result_2017[1:]]))

Conservatives:  317 Everyone Else:  333


## (Add tables for 2010 and 2015...)

In [22]:
for year in 2010, 2015:
    mps = getWikiMPs(year)
    table = '_'.join(['ge',str(year)])
    election_cursor.execute(textTableCreator(table))
    pushWikiMPs(election_db, election_cursor, table, mps)

In [23]:
election_cursor.execute('SELECT * FROM ge_2010')
election_cursor.fetchall()[0:3]

[(0, 'Aberavon', 'Labour', 'L', 'Hywel Francis', '', 'Seat held'),
 (1, 'Aberconwy', 'Labour', 'C', 'Guto Bebb', '', 'New constituency'),
 (2, 'Aberdeen North', 'Labour', 'L', 'Frank Doran', '', 'Seat held')]

# [Joins!](https://www.tutorialspoint.com/sqlite/sqlite_using_joins.htm)
## Combine the 2010 and 2015 tables.
## Each row has the same consituency and member_returned.

In [24]:
election_cursor.execute('''SELECT * FROM ge_2010 JOIN ge_2015 
    USING (constituency, member_returned)''')
election_cursor.fetchall()[1]

(7,
 'Altrincham and Sale West',
 'Conservative',
 'C',
 'Graham Brady',
 '',
 'Seat held',
 7,
 'Conservative',
 'C',
 '',
 'Seat held')

## Most column names are duplicated.
## Refer to them via the original tables.

In [25]:
election_cursor.execute('''SELECT ge_2010.member_party, ge_2015.member_party
    FROM ge_2010 JOIN ge_2015
    USING (constituency, member_returned)''')
election_cursor.fetchall()[1]

('C', 'C')

## Use the joined table to see who switched party between 2010 and 2015.

In [26]:
election_cursor.execute('''SELECT ge_2010.member_party, ge_2015.member_party,
    member_returned, constituency
    FROM ge_2010 JOIN ge_2015 USING (constituency, member_returned)
    WHERE NOT ge_2010.member_party = ge_2015.member_party''')
election_cursor.fetchall()

[('L', 'L Co-op', 'John Woodcock', 'Barrow and Furness'),
 ('C', 'UKIP', 'Douglas Carswell', 'Clacton'),
 ('L', 'L Co-op', 'Meg Hillier', 'Hackney South and Shoreditch'),
 ('L', 'L Co-op', 'Gareth Thomas', 'Harrow West'),
 ('L', 'L Co-op', 'Barry Sheerman', 'Huddersfield'),
 ('L', 'L Co-op', 'Mike Gapes', 'Ilford South'),
 ('L', 'L Co-op', 'Chris Evans', 'Islwyn'),
 ('L', 'L Co-op', 'Gavin Shuker', 'Luton South'),
 ('L', 'L Co-op', 'Chris Leslie', 'Nottingham East'),
 ('L', 'L Co-op', 'Mark Hendrick', 'Preston'),
 ('L', 'L Co-op', 'Jonathan Reynolds', 'Stalybridge and Hyde'),
 ('L', 'L Co-op', 'Geraint Davies', 'Swansea West'),
 ('L', 'L Co-op', 'Stella Creasy', 'Walthamstow'),
 ('L', 'L Co-op', 'Adrian Bailey', 'West Bromwich West')]

# (What happened to Douglas Carswell in 2017?)

In [27]:
election_cursor.execute('''SELECT constituency FROM ge_2017
    WHERE member_returned = "Douglas Carswell"''')
print(election_cursor.fetchall(), u'\U0001f604')

[] 😄


# Virtuosity with lists:
![Primitive](https://github.com/kingsBSD/sqlite-example/blob/master/images/brain_0.png?raw=true)
## Primitive:

In [28]:
silly_list = []
for n in range(18,31):
    silly_list.append(n//2 + 7)
silly_list

[16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22]

![Better](https://github.com/kingsBSD/sqlite-example/blob/master/images/brain_1.png?raw=true)
## Better: (list comprehensions)

In [29]:
[n//2 + 7 for n in range(18,31)]

[16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22]

![Better Still](https://github.com/kingsBSD/sqlite-example/blob/master/images/brain_3.png?raw=true)
## Better Still? (map)

In [30]:
def silly_function(n):
    return n//2 + 7
list(map(silly_function,range(18,31)))

[16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22]

![Genius!](https://github.com/kingsBSD/sqlite-example/blob/master/images/brain_4.png?raw=true)
## Genius! (map + lambda)

In [31]:
list(map(lambda x: x//2 + 7, range(18,31)))

[16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22]

# Map / List Comprehensions Considered Useful:

In [32]:
election_cursor.execute('SELECT DISTINCT incumbent_party FROM ge_2010')
list(map(lambda x: x[0],election_cursor.fetchall()))

['Labour',
 'Conservative',
 'SNP',
 'Liberal Democrats',
 'DUP',
 'SDLP',
 'Sinn Féin',
 'Respect',
 'BGPV',
 'Plaid Cymru',
 'Labour Party',
 'Scottish National Party',
 'UUP',
 'Liberal Democrat',
 'Democratic Unionist Party',
 'Conservative Party',
 'Health Concern']

## There are also dict comprehensions.

In [33]:
{i+1:day for i,day in enumerate(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])}

{1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat', 7: 'Sun'}

# Zip is fun:

In [34]:
english = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
deutsch = ['Mon', 'Die', 'Mit', 'Don', 'Fri', 'Sam', 'Son']
francais = ['Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim']
days = list(zip(english, deutsch, francais))
days

[('Mon', 'Mon', 'Lun'),
 ('Tue', 'Die', 'Mar'),
 ('Wed', 'Mit', 'Mer'),
 ('Thu', 'Don', 'Jeu'),
 ('Fri', 'Fri', 'Ven'),
 ('Sat', 'Sam', 'Sam'),
 ('Sun', 'Son', 'Dim')]

In [35]:
list(zip(*days))

[('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'),
 ('Mon', 'Die', 'Mit', 'Don', 'Fri', 'Sam', 'Son'),
 ('Lun', 'Mar', 'Mer', 'Jeu', 'Ven', 'Sam', 'Dim')]